# Extracting Data


## Introduction

In this chapter, you'll learn about some different ways to extract data: from the web, from documents, and elsewhere. This chapter uses packages such as **pandas-datareader** and **BeautifulSoup** that you may need to install first.

### Imports

First we need to import the packages we'll be using

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
# Set max rows displayed for readability
pd.set_option("display.max_rows", 6)
# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)

## Extracting data from files on the internet

As you will have seen in some of the examples in this book, it's easy to read data from the internet once you have the url and file type. Here, for instance, is an example that reads in the 'storms' dataset (we'll only grab the first 10 rows):

In [ ]:
pd.read_csv(
    "https://vincentarelbundock.github.io/Rdatasets/csv/dplyr/storms.csv", nrows=10
)

## Extracting data using APIs

Using an API (application programming interface) is another way to draw down information from the interweb. Their just a way for one tool, say Python, to speak to another tool, say a server, and usefully exchange information. The classic use case would be to post a request for data that fits a certain query via an API and to get a download of that data back in return. (You should always preferentially use an API over webscraping a site.)

Because they are designed to work with any tool, you don't actually need a programming language to interact with an API, it's just a *lot* easier if you do.

```{note}
An API key is needed in order to access some APIs. Sometimes all you need to do is register with site, in other cases you may have to pay for access.
```

 To see this, let's directly use an API to get some time series data. We will make the call out to the internet using the **requests** package.

An API has an 'endpoint', the base url, and then a URL that encodes the question. Let's see an example with the ONS API for which the endpoint is "https://api.ons.gov.uk/". The rest of the API has the form 'key/value', for example we'll ask for timeseries data 'timeseries' followed by 'JP9Z' for the vacancies in the UK services sector. We then ask for 'dataset' followed by 'UNEM' to specify which overarching dataset the series we want is in. The last part asks for the data with 'data'. Often you won't need to know all of these details, but it's useful to see a detailed example.

The data that are returned by APIs are typically in JSON format, which looks a lot like a nested Python dictionary and its entries can be accessed in the same way--this is what is happening when getting the series' title in the example below. JSON is not good for analysis, so we'll use **pandas** to put the data into shape.

In [ ]:
import requests

url = "https://api.ons.gov.uk/timeseries/JP9Z/dataset/UNEM/data"

# Get the data from the ONS API:
json_data = requests.get(url).json()

# Prep the data for a quick plot
title = json_data["description"]["title"]
df = (
    pd.DataFrame(pd.json_normalize(json_data["months"]))
    .assign(
        date=lambda x: pd.to_datetime(x["date"]),
        value=lambda x: pd.to_numeric(x["value"]),
    )
    .set_index("date")
)

df["value"].plot(title=title, ylim=(0, df["value"].max() * 1.2), lw=3.0);

We've talked about *reading* APIs. You can also create your own to serve up data, models, whatever you like! This is an advanced topic and we won't cover it; but if you do need to, the simplest way is to use [Fast API](https://fastapi.tiangolo.com/). You can find some short video tutorials for Fast API [here](https://calmcode.io/fastapi/hello-world.html).

### An easier way to interact with (some) APIs

Although it didn't take much code to get the ONS data, it would be even better if it was just a single line, wouldn't it? Fortunately there are some packages out there that make this easy, but it does depend on the API (and APIs come and go over time).

By far the most comprehensive library for accessing extra APIs is [**pandas-datareader**](https://pandas-datareader.readthedocs.io/en/latest/), which provides convenient access to:

- FRED
- Quandl
- World Bank
- OECD
- Eurostat

and more.

Let's see an example using FRED (the Federal Reserve Bank of St. Louis' economic data library). This time, let's look at the UK unemployment rate:

In [ ]:
import pandas_datareader.data as web

df_u = web.DataReader("LRHUTTTTGBM156S", "fred")

df_u.plot(title="UK unemployment (percent)", legend=False, ylim=(2, 6), lw=3.0);

## Webscraping

Webscraping is a way of grabbing information from the internet that was intended to be displayed in a browser. But it should only be used as a last resort, and only then when permitted by the terms and conditions of a website.

If you're getting data from the internet, it's much better to use an API whenever you can: grabbing information in a structure way is *exactly* why APIs exist. APIs should also be more stable than websites, which may change frequently. Typically, if an organisation is happy for you to grab their data, they will have made an API expressly for that purpose. I'd say it's pretty rare that there's a major website which *does* permit webscraping but which doesn't have an API; for these websites, if they don't have an API, chances scraping is against their terms and conditions. Those terms and conditions may be enforceable by law (different rules in different countries here, and you really need legal advice if it's not unambiguous as to whether you can scrape or not.)

There are other reasons why webscraping is not so good; for example, if you need a back-run then it might be offered through an API but not shown on the webpage. (Or it might not be available at all, in which case it's best to get in touch with the organisation or check out WaybackMachine in case they took snapshots).

So I'm pretty down on webscraping as there's almost always a better solution. However, there are occasionally times when it is useful.

If you do find yourself in a scraping situation, be really sure to check that's legally allowed and also that you are not violating the website's `robots.txt` rules: this is a special file on almost every website that sets out what's fair play to crawl (conditional on legality) and what robots should not go poking around in.

In Python, you are spoiled for choice when it comes to webscraping. There are five very strong libraries that cover a real range of user styles and needs: **requests**, **lxml**, **beautifulsoup**, **selenium**, and *scrapy**.

For quick and simple webscraping, my usual combo would **requests**, which does little more than go and grab the HTML of a webpage, and **beautifulsoup**, which then helps you to navigate the structure of the page and pull out what you're actually interested in. For dynamic webpages that use javascript rather than just HTML, you'll need **selenium**. To scale up and hit thousands of webpages in an efficient way, you might try **scrapy**, which can work with the other tools and handle multiple sessions, and all other kinds of bells and whistles... it's actually a "web scraping framework".

It's always helpful to see coding in practice, so that's what we'll do now, but note that we'll be skipping over a lot of important detail such as user agents, being 'polite' with your scraping requests, being efficient with caching and crawling.

In lieu of a better example, let's scrape [http://aeturrell.com/](http://aeturrell.com/)

In [ ]:
url = "http://aeturrell.com/"
page = requests.get(url)
page.text[:300]

Okay, what just happened? We asked requests to grab the HTML of the webpage and then printed the first 300 characters of the text that it found.

Let's now parse this into something humans can read (or can read more easily) using beautifulsoup:

In [ ]:
soup = BeautifulSoup(page.text, "html.parser")
print(soup.prettify()[500:1000])

Now we see more structure of the page and even some *HTML tags* such as 'title' and 'link'. Now we come to the data extraction part: say we want to pull out every paragraph of text, we can use beautifulsoup to skim down the HTML structure and pull out only those parts with the paragraph tag ('p').


In [ ]:
# Get all paragraphs
all_paras = soup.find_all("p")
# Just show one of the paras
all_paras[52]

To make this more readable, you can use the `.text` method:

In [ ]:
all_paras[52].text

Now let's say we didn't care about most of the page, we *only* wanted to get hold of the names of projects. For this we need to identify the tag type of the element we're interested in, in this case 'div', and it's class type, in this case "project-name". We do it like this (and show nice text in the process):


In [ ]:
projects = soup.find_all("div", class_="project-name")
projects = [x.text.strip() for x in projects]
projects

Hooray! We managed to get the information we wanted: all we needed to know was the right tags. A good tip for finding the tags of the info you want is to look at in your browser (eg Google Chrome) and then right-click on the bit you're interested in, then hit 'Inspect'. This will show you the HTML element of the bit of the page you clicked on.

That's almost it for this very, very brief introduction to webscraping. We'll just see one more thing: how to iterate over multiple pages.

Imagine we had a root webpage such as "www.codingforeconomists.com" which had subpages such as "www.codingforeconomists.com/page=1", "www.codingforeconomists.com/page=2", and so on. One need only iterate create the HTML strings to pass into a function that scrapes each one and return the relevant data, eg for the first 50 pages, and with a function called `scraper`, one might run

```
start, stop = 0, 50
root_url = "www.codingforeconomists.com/page="
info_on_pages = [scraper(root_url + str(i)) for i in range(start, stop)]
```

That's all we'll cover here but remember we've barely *scraped* the surface of this big, complex topic. If you want to read about an application, it's hard not to recommend the paper on webscraping that has undoubtedly change the world the most, and very likely has affected your own life in numerous ways: ["The PageRank Citation Ranking: Bringing Order to the Web"](http://ilpubs.stanford.edu:8090/422/) by Page, Brin, Motwani and Winograd. For a more in-depth example of webscraping, check out realpython's [tutorial](https://realpython.com/python-web-scraping-practical-introduction/).

### Webscraping Tables

Often there are times when you don't actually want to scrape an entire webpage and all you want is the data from a *table* within the page. Fortunately, there is an easy way to scrape individual tables using the **pandas** package.

We will read data from the first table on 'https://simple.wikipedia.org/wiki/FIFA_World_Cup' using **pandas**. The function we'll use is `read_html`, which returns a list of dataframes of all the tables it finds when you pass it a URL. If you want to filter the list of tables, use the `match=` keyword argument with text that only appears in the table(s) you're interested in.

The example below shows how this works; looking at the website, we can see that the table we're interested in (of past world cup results), has a 'fourth place' column while other tables on the page do not. Therefore we run:

In [ ]:
df_list = pd.read_html(
    "https://simple.wikipedia.org/wiki/FIFA_World_Cup", match="Fourth Place"
)
# Retrieve first and only entry from list of dataframes
df = df_list[0]
df.head()

This gives us the table neatly loaded into a **pandas** dataframe ready for further use.

## Extracting data from PDFs

PDFs are great. Unfortunately, some people love them so much that they think they're an appropriate way to store data rather than a convenient way to share text and/or figures. Or perhaps there's a table in a PDF that you'd legitimately like to get the info out from.


### Extracting images and text from PDFs

We'll use [**pdfminer.six**](https://pdfminersix.readthedocs.io/en/latest/) to get text out of the same PDF. For this simple operation, it's just a single command:

In [ ]:
from pathlib import Path
from pdfminer.high_level import extract_text

# Download the pdf_with_table.pdf file from
# https://github.com/aeturrell/coding-for-economists/blob/main/data/pdf_with_table.pdf
# and put it in a subfolder called data before running the next line
text = extract_text(Path("data/pdf_with_table.pdf"))

# Show only first 200 characters of text:
print(text[:200])

This uses the 'high-level' part of the library, which is designed to help cover the most common use cases. But there's a lot more flexibility there if you need it: essentially, **pdfminer** will break up a pdf into a series of elements that you can sift through in the much the same way as **beautifulsoup** does for HTML. For example, you can use library to [extract images](https://pdfminersix.readthedocs.io/en/latest/howto/images.html) from PDFs too.

### Tables

The single best solution to grab tables is probably [**camelot**](https://camelot-py.readthedocs.io/en/master/index.html). Note that it does need you to have Ghostscript installed on your computer; you can find more information about the dependencies [here](https://camelot-py.readthedocs.io/en/master/user/install.html). It only works with text-based PDFs and not scanned documents: basically, if you can click and drag to select text in your table in a PDF viewer, then your PDF is text-based. In that case, **camelot** is able to sift through the contents and grab any tables and then pass them back as csvs or even **pandas** dataframes.

At the time of writing, **camelot** had some versioning issues related to a dependency on an outdated version of **sqlalchemy**. You may need to install it in a separate virtual environment to use it.

Here's a small example that assumes you have a pdf with a table in stored in a local directory:

```python
import camelot
# Grab the pdf
tables = camelot.read_pdf(os.path.join('data', 'pdf_with_table.pdf'))
```

To extract any of the $n$ tables that are retrieved into a pandas dataframe individually, use `tables[0].df`. 

Note that **camelot** is not perfect--so it can produce a report on how it did when it tried to extract each table, which includes an accuracy score. This is found using, for example, `tables[0].parsing_report`.


### Other useful PDF packages

[**pdfcomments**](https://github.com/hoffmangroup/pdfcomments) is a library that allows you to strip out comments and sticky notes from PDF. This need not strictly be about data extraction, but [**PyPDF2**](https://pythonhosted.org/PyPDF2/) allows you to both split a PDF into separate pages and merge multiple PDFs together; see [here](http://www.blog.pythonlibrary.org/2018/04/11/splitting-and-merging-pdfs-with-python/) for the steps. (You can drag and drop pages using preview on MacOS but this library may be the easiest way to do the same thing on Windows.)

## Text extraction

Not everything is a PDF file! If you want to get the text out of .doc, .docx, .epub, .gif, .json, .jpg, *.mp3*, .odt, .pptx, .rtf, .xlsx, .xls and, actually, .pdf too, then **textract** is for you. Mostly, it's a wrapper around a ton of other libraries. The upside is that getting the text out should be as easy as calling

```python
import textract
text = textract.process(Path('path/to/file.extension'))
```

The downside is that it requires that some other (non-Python) libraries be installed and it doesn't (yet) work on Windows.

## Review

If you know how to get data from:

- ✅ the internet using a URL;
- ✅ the internet using an API;
- ✅ the internet using webscraping; and
- ✅ PDFs, Microsoft Word Documents, and more, using tools like **pdfminer.six** and **textract**

then you have a good basic set of tools for getting the data that you need into a form you can use.